# Lektion 8 - ML pipelines: Struktur och automatisering

**Assignment: Build a minimal training pipeline**

Instructions:
1. Create a small end-to-end pipeline in code
2. Save outputs and metrics
3. Keep short comments explaining design choices

## Task 1: Pipeline in code
Build a small end-to-end pipeline with preprocessing and a model.

In [1]:
# TODO: Build a scikit-learn Pipeline with:
# - StandardScaler
# - Model of choice (LogisticRegression or SVC)

# En pipeline är en serie av steg som vi kör
# Inom ML, använder vi ofta pipelines för preprocessingsteg
# som till exempel standardisering, transformering och reshaping, 
# men även för träning.
# 
# Idag bygger vi en pipeline med standardisering och modellskapande
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

# Nedan bygger vi en pipeline.
# Pipelinen som vi har byggt sätter ihop både vårt preprocessingsteg
# och skapandet av vår modell till en körning.
# Det blir då väldigt enkelt att återskapa samma flöde
pipeline = Pipeline(
    steps = [
        ("scaler", StandardScaler()),
        ("model", LogisticRegression(max_iter=1000))
    ]
)




In [2]:
# TODO: Train and evaluate on a dataset
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

data = load_iris(as_frame=True)
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.25, random_state=42)


In [6]:
print("X_test shape:", X_test.shape)
print("X_train shape:", X_train.shape)

X_test shape: (38, 4)
X_train shape: (112, 4)


# Se L4_assignment_CLASSROOM.ipynb för utförlig EDA:

[L4_assignment_CLASSROOM.ipynb](../L4/L4_assignment_CLASSROOM.ipynb)

[Web version](https://github.com/AndreasFurth/ML-Frameworks/blob/main/L4/L4_assignment_CLASSROOM.ipynb)

In [8]:
# The training:


# Med vår pipeline så tränar vi, och sedan utvärderar vår modell

pipeline.fit(X_train,y_train)
preds = pipeline.predict(X_test)

metrics = {
    "accuracy": accuracy_score(y_test, preds),
    "f1_macro": f1_score(y_test, preds, average="macro")
}

print(metrics)


{'accuracy': 1.0, 'f1_macro': 1.0}


## Task 2: Automate training
Wrap the workflow into a reusable experiment function.

In [ ]:
# TODO: Wrap training in a function run_experiment(config)

# En sån här funktion riskerar att bli väldigt lång
# Vi måste gå balansgång mellan generaliserbarhet 
# och korthet/läsbarhet.
# Om idéen är att kunna återanvända vår experimentfunktion
# så är lång längd ett piller som vi kan behöva svälja
def run_experiment(config = {"scaler": StandardScaler(), "model": LogisticRegression(max_iter=1000), "params": None}):
    
    # Nedan är en generaliserbar pipeline
    # MEN, den är kanske inte maximalt användbar,
    # eftersom användaren tvingas hålla koll på 
    # och skicka in alla separata steg själv
    pipeline = Pipeline(
        steps= [
            ("scaler", config["scaler"]),
            ("model", config["model"]),
            ("params", config["params"])
        ]
    )

def run_experiment_not_generalised(config):
    # Use a small dataset to keep runtime low
    iris = load_iris()
    X_train, X_test, y_train, y_test = train_test_split(
        iris.data, iris.target, test_size=0.2, random_state=42, stratify=iris.target
    )

    pipeline = Pipeline(
        steps=[
            ("scaler", StandardScaler()),
            ("model", LogisticRegression(max_iter=config["max_iter"], C=config["C"])),
        ]
    )

    pipeline.fit(X_train, y_train)
    preds = pipeline.predict(X_test)

    metrics = {
        "accuracy": accuracy_score(y_test, preds),
        "f1_macro": f1_score(y_test, preds, average="macro"),
        "params": {"C": config["C"], "max_iter": config["max_iter"]},
    }

    # Save outputs
    ("metrics.json").write_text(json.dumps(metrics, indent=2))
    joblib.dump(pipeline, "model.joblib")

    return metrics



In [9]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import get_scorer

def run_experiment(config):
    """
    Run a general ML experiment entirely from a config dict.

    config: dict
        {
            "X": feature matrix,
            "y": target vector,
            "test_size": float (optional, default=0.2),
            "random_state": int (optional, default=42),
            "preprocessing": list of (name, transformer) tuples (optional),
            "model": sklearn estimator,
            "params": dict of hyperparameters for GridSearchCV (optional),
            "scoring": str or callable metric (optional, default='accuracy')
        }
    """
    
    # Extract from config with defaults
    # Vi hämtar de relevanta värdena från config
    X = config["X"]
    y = config["y"]
    test_size = config.get("test_size", 0.2)
    random_state = config.get("random_state", 42)
    preprocessing = config.get("preprocessing", [])
    model = config["model"]
    params = config.get("params", None)
    scoring = config.get("scoring", "accuracy")
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state
    )
    
    # Build pipeline
    # Nedan så tar bygger i en lista av steps
    # först är all preprocessing (scaling, transforms etc)
    # och sist är modellen.
    steps = preprocessing + [("model", model)] 

    # steps ser typiskt ut ungefär såhär
#    steps= [
#             ("scaler", config["scaler"]),
#             ("model", config["model"]), 
#        ]

    pipeline = Pipeline(steps=steps)
    
    # Wrap with GridSearchCV if params provided
    # ÖVERKURS: kika själva om ni är nyfikna
    if params:
        pipeline = GridSearchCV(pipeline, params, cv=5, n_jobs=-1, scoring=scoring)
    
    # Fit
    pipeline.fit(X_train, y_train)
    
    # Evaluate
    # Här använder vi sklearns get_scorer() för att räkna ut vår performance
    # tidigare har vi i regel importerat en specifik metric (f1-score, accuracy)
    # Vi brukar också köra confusion matrix, det kan vi göra senare
    scorer = get_scorer(scoring)
    score = scorer(pipeline, X_test, y_test)
    
    print(f"{scoring} on test set: {score:.4f}")
    
    return pipeline, score

In [17]:
# TODO: Save metrics to metrics.json

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.svm import SVC
import joblib
import json

X, y = load_iris(return_X_y=True)

configs = []
config_LogReg_025 = {
    "config_name": "LogReg_025",
    "X": X,
    "y": y,
    "test_size": 0.25,
    "random_state": 123,
    "preprocessing": [
        ("scaler", StandardScaler()),
        ("pca", PCA(n_components=2))
    ],
    "model": LogisticRegression(max_iter=1000),
    "params": {
        "model__C": [0.1, 1, 10],
    },
    "scoring": "accuracy"
}

config_SVM_03 = {
    "config_name": "SVM_03",
    "X": X,
    "y": y,
    "test_size": 0.3,
    "random_state": 456,
    "preprocessing": [
        ("scaler", StandardScaler())
    ],
    "model": SVC(),
    "params": {
        "model__C": [0.1, 1, 10],
        "model__kernel": ["linear", "rbf"]
    },
    "scoring": "accuracy"
}

# create a config for a third experiment with different preprocessing and model
config_LogReg_02 = {
    "config_name": "LogReg_02",
    "X": X,
    "y": y,
    "test_size": 0.2,
    "random_state": 789,
    "preprocessing": [
        ("scaler", StandardScaler())
    ],
    "model": LogisticRegression(max_iter=1000),
    "params": {
        "model__C": [0.01, 0.1, 1],
    },
    "scoring": "accuracy"
}

configs = [config_LogReg_025, config_SVM_03, config_LogReg_02]

for config in configs:

    pipeline, score = run_experiment(config)

    # Save outputs
    metrics = {
        "accuracy": score,
        "params": pipeline.best_params_ if hasattr(pipeline, "best_params_") else None
    }

    # save our metrics to a json file
    with open(f"metrics_{config['config_name']}.json", "w") as f:
        json.dump(metrics, f, indent=2) 

    # save our model to a joblib file
    joblib.dump(pipeline, f"model_{config['config_name']}.joblib")

accuracy on test set: 0.8421
accuracy on test set: 0.9778
accuracy on test set: 0.9667


In [13]:
# TODO: Save the trained model with joblib

joblib.dump(pipeline, "model.joblib")

['model.joblib']

## Task 3: CLI (optional but recommended)
Parameterize runs and log chosen values.

In [ ]:
# TODO: Use argparse to pass model params (e.g., C, max_iter)

In [ ]:
# TODO: Log chosen params into metrics.json

In [ ]:
print("Done! You created a small reproducible ML pipeline.")